只使用close预测
找到最佳参数

In [1]:
import os
while os.getcwd().split('\\')[-1] != "Data_Analyst_FJNU":
    os.chdir('..')
print(os.getcwd())

e:\Desktop\学业\第五学期课业\数据分析\Data_Analyst_FJNU


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
def get_stock_data(code):
    stock_data =  pd.read_csv(f'./Data/history_A_stock_k_data/{code}.csv')
    stock_data = stock_data[['date','close']]
    stock_data['date'] = pd.to_datetime(stock_data['date'])
    stock_data.set_index('date',inplace=True)
    return stock_data

def plot_dataframe(df):
    plt.plot(df.index, df['close'])
    plt.xlabel('Date')
    plt.ylabel('Close')
    plt.title('DataFrame Plot')
    plt.show()
    
def create_sequences(data, sequence_length):
    X, y, indices = [], [], []
    for i in range(len(data) - sequence_length):
        X.append(data.iloc[i:i+sequence_length])
        y.append(data.iloc[i+sequence_length])
        indices.append(data.index[i+sequence_length])
    return np.array(X), np.array(y), indices

def build_model(units, LSTM_layers, Dense_layers, X_train, y_train):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1],1)))
    model.add(Dropout(0.1))
    
    for _ in range(LSTM_layers):
        model.add(LSTM(units=units, return_sequences=True))
        model.add(Dropout(0.1))
    
    model.add(LSTM(units=units))
    model.add(Dropout(0.1))
    
    for _ in range(Dense_layers):
        model.add(Dense(units=units, activation='relu'))
        model.add(Dropout(0.1))
    
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
    model.summary()

    return model


In [4]:
# dimension = 50
# units = 32
# LSTM_layers = 2
# Dense_layers = 2
code = 'sh.600019'
model_dir = f'./Data/model/{code}'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
dimension_list = [15, 30, 50]
units_list = [16, 32, 64]
LSTM_layers_list = [1,2,3]
Dense_layers_list = [1,2,3]
df = get_stock_data('sh.600019')
for dimension in dimension_list:
    X, y, indices = create_sequences(df['close'], dimension)
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X)
    y = scaler.fit_transform(y.reshape(-1, 1))
    X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(X, y, indices, test_size=0.1, random_state=42, shuffle=False)
    for units in units_list:
        for LSTM_layers in LSTM_layers_list:
            for Dense_layers in Dense_layers_list:
                model = build_model(units, LSTM_layers, Dense_layers,X_train, y_train)
                early_stopping = EarlyStopping(
                    monitor='val_loss', 
                    patience=10, 
                    restore_best_weights=True
                )
                model_file = f"{model_dir}/"+"{val_loss:.8f}_{epoch:02d}"+f"_dim_{dimension}_units_{units}_lstm_{LSTM_layers}_dense_{Dense_layers}.h5"
                print(model_file)
                checkpoint = ModelCheckpoint(
                    filepath = model_file,
                    save_weights_only=True,
                    monitor='val_loss',        
                    save_best_only=True,         
                    mode='min',       
                )
                history = model.fit(X_train, y_train, epochs=40, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping], verbose=1)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 16)            1152      
                                                                 
 dropout (Dropout)           (None, 15, 16)            0         
                                                                 
 lstm_1 (LSTM)               (None, 15, 16)            2112      
                                                                 
 dropout_1 (Dropout)         (None, 15, 16)            0         
                                                                 
 lstm_2 (LSTM)               (None, 16)                2112      
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 16)               

In [5]:
def evaludate(model,X,y):
    loss, mean_absolute_error = model.evaluate(X,y)
    print(f'Test Loss: {loss}, Test Mean Absolute Error: {mean_absolute_error}')
    # Plot training history
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
# evaludate(model,X_test,y_test)

In [6]:
def predict_show(X_test, indices):
    y_pred = model.predict(X_test)
    y_pred_actual = scaler.inverse_transform(y_pred)

    result_df = pd.DataFrame({'Date': indices, 'Predicted': y_pred_actual.flatten()})
    result_df.index = pd.to_datetime(result_df.index)
    result_df.set_index('Date', inplace=True)
    result_df.sort_index(inplace=True)

    # plot
    # scatter

    plt.figure(figsize=(12, 6))
    plt.plot(result_df.index, result_df['Predicted'], label='Predicted', color='red')
    plt.plot(df.index, df['close'], label='Actual', color='blue')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Actual vs Predicted Stock Prices')
    plt.legend()
    plt.show()


    recent_month_data = result_df[result_df.index >= result_df.index.max() - pd.DateOffset(months=12)]
    plt.figure(figsize=(12, 6))
    plt.plot(recent_month_data.index, recent_month_data['Predicted'], label='Predicted', color='red')
    plt.plot(df.index[-30*12:], df['close'].tail(30*12), label='Actual', color='blue')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title('Actual vs Predicted Stock Prices (Most Recent Month)')
    plt.legend()
    plt.show()

    print(result_df.tail(5))
# predict_show(X_test, test_indices)

In [7]:
# predict_show(X[-1098:],df.index[-1098:])

In [8]:
# predict_show(X,df.index[dimension:])

In [9]:
# predict_show(X[-2:],df.index[-2:])

In [10]:
def predict_next(model, df):
    last_sequence = df.iloc[-dimension:].values
    last_sequence = scaler.transform(last_sequence)
    last_sequence = last_sequence.transpose()
    predicted_value = model.predict(last_sequence)
    predicted_value = scaler.inverse_transform(predicted_value)
    last_date = df.index[-1]
    result_data = {'close': predicted_value[0]}
    result_date = last_date + pd.DateOffset(days=1)
    result_df = pd.DataFrame(result_data, index=[result_date])
    new_df = pd.concat([df, result_df], axis=0)
    return new_df
# new_df = df
# new_df = predict_next(model, new_df)
# new_df.tail()
# new_df = predict_next(model, new_df)
# new_df.tail()
# new_df = predict_next(model, new_df)
# new_df.tail()
# new_df = predict_next(model, new_df)
# new_df.tail()